## Project Overview 

###  Goals: To find drivers of churn.

Why? 

To help idenitfy and potentialy retain profitable customers that might potentialy leave Telco as thier communcaitions service provider.

Acquire Data

In [1]:
# imports 
import pandas as pd
import numpy as np

# my own files with my own functions
import acquire
import prepare

# train test split function
from sklearn.model_selection import train_test_split

Acquire and Prep Data to split 

In [2]:
df = prepare.prep_telco(acquire.get_telco_data())

Split Data into train, evaluate, and test, stratisfying on the target, and Explore only the train data

In [3]:
train, evaluate, test = prepare.split_data(df, 'churn')

train: 4225 (60.0% of 7043)
validate: 1409 (20.0% of 7043)
test: 1409 (20.0% of 7043)


First Question

In [ ]:
(do not present content that is not goingt to be in your presentation.)

Percent of customers who churn

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming you have a DataFrame with a 'churn' column (0 for no churn, 1 for churn)
data = {'churn': [0, 1, 1, 0, 0, 1, 0, 1, 0, 0]}
df = pd.DataFrame(data)

# Count the number of customers who have churned and those who haven't
churn_counts = df['churn'].value_counts()

# Plot a pie chart
labels = ['Not Churned', 'Churned']
colors = ['lightblue', 'lightcoral']

plt.pie(churn_counts, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Distribution of Churned vs Not Churned Customers')
plt.show()

You have to use markdown cell to go over the steps you took to prepare the data and why you made the decsions you made


Data Preparation:
    Data types changed:
        I changed the data type for Total Charges because it was non-numerical.
        Also, the 
    Data null values:
        I changed null values in total charges to zero. why? give deicsions and reasons. was this done before or after you split. 
        I changed the null values in internet service type to "No internet service" so it can match the rest of the columns that have no internet service. if you look at the values in the other columns where it says no internet service is the same place where internet service type is null. 
 